In [3]:
import logging
import sys

# Configurations
from configuration.llm_inference_configuration import APILLMConfiguration
from configuration.embedding_inference_configuration import APIEmbeddingModelConfiguration, LocalEmbeddingModelConfiguration
from configuration.nlp_configuration import NLPConfiguration
from configuration.tool_configuration import ToolConfiguration, DemonstrationSamplingToolConfiguration
from configuration.operator_configuration import CoTOperatorConfiguration
# System components
from llm.request_llm import RequestLLM
from embedding.request_embedding import RequestEmbeddingModel
from nlp.spacy_nlp import SpacyNLP
from tools.python_code_runner import PythonCodeRunnerTool
from tools.demonstration_sampling import DemonstrationSamplingTool
# Prompt
from prompt.zero_shot import ZeroShotPrompt
from prompt.few_shot import FewShotPrompt
from prompt.user_message import UserMessagePrompt
from prompt.assistant_message import AssistantMessagePrompt
from prompt.tool_message import ToolMessagePrompt
# Memory
from base_classes.memory.memory_atom import AbstractMemoryAtom
from base_classes.memory.memory_block import AbstractMemoryBlock
from base_classes.memory.memory_topic import AbstractMemoryTopic
from base_classes.memory.datatypes.data_item import PromptDataItem
from base_classes.memory.memory_worker import MemoryWorker
# MCP
# from tools.mcp_server
# Operator
from base_classes.operator import AbstractOperator
from operators.cot import CoTOperator
from operators.debate import DebateOperator
from operators.react import ReactOperator

logging.basicConfig(
    level=logging.CRITICAL,
    format='%(asctime)s\t\t%(levelname)s\t%(message)s',
    handlers=[logging.StreamHandler()]
)

all_logger_names = list(logging.root.manager.loggerDict.keys())
if '' not in all_logger_names:
    all_logger_names.append('')

for logger_name in all_logger_names:
    logger = logging.getLogger(logger_name)
    if logger_name.startswith("Orbit"):
        logger.setLevel(logging.DEBUG)
    elif logger_name == '':
        logger.setLevel(logging.CRITICAL)
    else:
        logger.setLevel(logging.CRITICAL)


# 1. System component test

## 1.1. LLM test

### 1.1.1. Azure OpenAI test

In [2]:
gpt_4o_azure_configuration = APILLMConfiguration()
gpt_4o_azure_configuration.load("configuration/yaml/llm/azure_gpt_4o.yaml")
azure_gpt_4o = RequestLLM(gpt_4o_azure_configuration)

try:
    azure_gpt_4o_1 = RequestLLM(gpt_4o_azure_configuration)
except Exception as e:
    logging.info("LLM unique ID test successful")
    logging.error(f"Failed to create AzureOpenAILLM instance: {e}")

In [ ]:
sample_zero_shot_message = ZeroShotPrompt(
    [
        {
            "role": "user", 
            "content": "Write a 1000-word very long report about economic growth in Vietnam"
        }
    ]
)

sample_zero_shot_message_responses = azure_gpt_4o.query(query=sample_zero_shot_message, num_responses=2)
print(azure_gpt_4o.get_response_texts(sample_zero_shot_message_responses))

In [ ]:
sample_few_shots_message = FewShotPrompt(
    [
        {
            "role": "system", 
            "content": "You are NetMind assistant. You task is to answer to the user anything about Viettel Group."
        },
        {
            "role": "user", 
            "content": "Tell me about Viettel Group."
        },
        {
            "role": "assistant", 
            "content": "Viettel Group is a Vietnamese multinational telecommunications company headquartered in Hanoi, Vietnam. It is a state-owned enterprise and operated by the Ministry of Defence. You can find out more about Viettel Group at https://viettel.vn/."
        },
        {
            "role": "user",
            "content": "What is the revenue of Viettel Group? Tell me the list of child companies of Viettel Group?"
        }
    ]
)

sample_few_shots_message_responses = azure_gpt_4o.query(query=sample_few_shots_message.prompt, num_responses=2)
print(azure_gpt_4o.get_response_texts(sample_few_shots_message_responses))

## 1.2. Embedding test

### 1.2.1. Request embedding model test

In [ ]:
text_embedding_ada_002_config = APIEmbeddingModelConfiguration()
text_embedding_ada_002_config.load("configuration/yaml/embedding/text-embedding-ada-002.yaml")
text_embedding_ada_002 = RequestEmbeddingModel(text_embedding_ada_002_config)

In [ ]:
# text_embedding_ada_002.encode("Hello world")
text_embedding_ada_002.similarity("Hello world", "Hello world")

## 1.3. Tool test

### 1.3.1. Python code runner tool

In [ ]:
python_code_runner_config = ToolConfiguration()
python_code_runner_config.load("configuration/yaml/tools/python_code_runner.yaml")
python_code_runner = PythonCodeRunnerTool(python_code_runner_config)

In [ ]:
input_code_list = [
    {
        "code_string": "a = 11\nb = 20\nresult = a + b\n\ndef greet(name):\n return f'Hello, {name}!'\n \nmessage = greet('World')",
        "id": "1"
    },
    {
        "code_string": "def multiply(x, y):\n\treturn x * y\n\nnum1 = 5\nnum2 = 3\nresult = multiply(num1, num2)\n\nprint(f'The product of {num1} and {num2} is {result}.')",
        "id": "2"
    }
]

results = await python_code_runner.execute(input_code_list=input_code_list)
results

### 1.3.2. Demostration sampling tool

In [ ]:
demonstration_sampling_tool_config = DemonstrationSamplingToolConfiguration()
demonstration_sampling_tool_config.load("configuration/yaml/tools/demonstration_sampling.yaml")
demonstration_sampling_tool = DemonstrationSamplingTool(tool_config=demonstration_sampling_tool_config)

In [ ]:
samples = await demonstration_sampling_tool.execute(input_message=UserMessagePrompt(
    prompt = [
        {
            "role": "user", 
            "content": "Organize a party"
        }
    ]
))
samples

In [ ]:
print(samples)

## 1.4. NLP model test 

### 1.4.1. SpaCy NLP test

In [ ]:
spacy_nlp_config = NLPConfiguration()
spacy_nlp_config.load("configuration/yaml/nlp/spacy.yaml")
spacy_model = SpacyNLP(spacy_nlp_config)

In [ ]:
spacy_model.lemmatize("Hello, my name is John Doe. I am a software engineer. I love coding and solving problems.")

# 2. Memory test

In [3]:
from test_utils.memory import *

In [ ]:
# Memory Worker Initiation
spacy_nlp_config = NLPConfiguration()
spacy_nlp_config.load("configuration/yaml/nlp/spacy.yaml")
spacy_model = SpacyNLP(spacy_nlp_config)

gpt_4o_azure_configuration = APILLMConfiguration()
gpt_4o_azure_configuration.load("configuration/yaml/llm/azure_gpt_4o.yaml")
azure_gpt_4o = RequestLLM(gpt_4o_azure_configuration)

text_embedding_ada_002_config = APIEmbeddingModelConfiguration()
text_embedding_ada_002_config.load("configuration/yaml/embedding/text-embedding-ada-002.yaml")
text_embedding_ada_002 = RequestEmbeddingModel(text_embedding_ada_002_config)

memory_worker = MemoryWorker(
    emb_model=text_embedding_ada_002,
    llm=azure_gpt_4o,
    nlp_model=spacy_model
)

In [ ]:
memory_worker.feature_engineer_for_memory_block(mem_block_1)
memory_worker.feature_engineer_for_memory_block(mem_block_2)
memory_worker.feature_engineer_for_memory_block(mem_block_3)
memory_worker.feature_engineer_for_memory_block(mem_block_4)
memory_worker.feature_engineer_for_memory_block(mem_block_5)
memory_worker.feature_engineer_for_memory_block(mem_block_6)

memory_worker.feature_engineer_for_memory_topic(mem_topic_1)
memory_worker.feature_engineer_for_memory_topic(mem_topic_2)
memory_worker.feature_engineer_for_memory_topic(mem_topic_3)

In [ ]:
new_query = [{
    'role': 'user',
    'content': 'I have a problem with my integer_adder() program. It can not output the sum of numbers given to it. Can you calculate the sum of integers from 1 to 100?'
}]
new_mem_atom = AbstractMemoryAtom(data=PromptDataItem(UserMessagePrompt(new_query)))

new_mem_block = AbstractMemoryBlock()
new_mem_block.add_memory_atom(new_mem_atom)

In [ ]:
relevant_topics = memory_worker.select_relevant_topics(new_mem_block, mem_stack, top_n=1)

In [ ]:
for mtp, scr in relevant_topics:
    mtp.add_mem_block(new_mem_block)

In [ ]:
retrieved_mem_blocks = memory_worker.memory_block_retrieval(new_mem_block)

In [ ]:
print(retrieved_mem_blocks[3])

# 3. Operator

## 3.1. CoT

In [2]:
gpt_4o_azure_configuration = APILLMConfiguration()
gpt_4o_azure_configuration.load("configuration/yaml/llm/azure_gpt_4o.yaml")
azure_gpt_4o = RequestLLM(gpt_4o_azure_configuration)

demonstration_sampling_tool_config = DemonstrationSamplingToolConfiguration()
demonstration_sampling_tool_config.load("configuration/yaml/tools/demonstration_sampling.yaml")
demonstration_sampling_tool = DemonstrationSamplingTool(tool_config=demonstration_sampling_tool_config)

cot_operator_config = CoTOperatorConfiguration()
cot_operator_config.load("configuration/yaml/operators/cot.yaml")
cot_operator = CoTOperator(config=cot_operator_config)

In [ ]:
cot_test_message = UserMessagePrompt(
    [
        {
            "role": "user", 
            "content": "Organize a party"
        }
    ]
)

cot_test_answer, cot_test_dependency_graph = await cot_operator.run(input_message=cot_test_message)

In [ ]:
print(cot_test_answer)

In [ ]:
prompt = UserMessagePrompt([{'role': 'user', 'content': '\nYou are tasked with creating a detailed step-by-step plan to address a specific problem. Each plan should clearly outline the necessary steps, ensuring clarity and logical flow. Use the following format to present your response:\n```\n<problem>[The problem description is given here]</problem>\n<plan>\n   <step id=1>\n      [The first step of the plan is described here.]\n   <\\step>\n   <step id=2>\n      [The second step of the plan is described here.]\n   <\\step>\n   <step id=3>\n      [The third step of the plan is described here.]\n   <\\step>\n   ...\n   <step id=[The step number is given here]>\n      [The nth step of the plan is described here.]\n   <\\step>\n<\\plan>\n```\n\n### Instructions:\n1. Provide a clear and concise description of the problem.\n2. Develop a comprehensive step-by-step plan, ensuring each step is labeled with a unique ID.\n3. Maintain logical progression in the steps to facilitate understanding and execution.\n<problem>Plan a birthday party for a 10-year-old.</problem>\n\n<plan>\n\n<step id=1>\nDecide on a theme for the party.\n<\\step>\n\n<step id=2>\nCreate a guest list of friends and family.\n<\\step>\n\n<step id=3>\nChoose a location for the party (home or venue).\n<\\step>\n\n<step id=4>\nSend out invitations at least two weeks in advance.\n<\\step>\n\n<step id=5>\nPlan games and activities suitable for the age group.\n<\\step>\n\n<step id=6>\nOrder or bake a birthday cake and other snacks.\n<\\step>\n\n<step id=7>\nDecorate the venue according to the chosen theme.\n<\\step>\n\n<step id=8>\nPrepare goodie bags for guests to take home.\n<\\step>\n\n<\\plan><problem>Organize a party</problem>. Please generate the plan following the given instruction.'}])


answer = azure_gpt_4o.query(query=prompt, num_responses=2)
print(answer)

In [ ]:
prompt = UserMessagePrompt([{'role': 'user', 'content': 'Generate a paragaph of 10 sentences. Enter a newline for each sentence'}])


answer = azure_gpt_4o.query(query=prompt, num_responses=2)
print(answer)